# Indices and Keys
Indices are structures created to optimize access to data. They are used to retrieve data from a relational database in an efficient way. Indices speeds up data retrieval in search and join operations in tables with many records, reducing the number of records that the engine reads to fetch the result. 

Keys are another concept somewhat related to indices. Keys special fields used to identify records within a table, they are a logical structure in our model primarily used to define relationships, whereas indices control the way the information is stored and retrieved in the database.

On the other hand, the performance of write operations (insert and specially update) is penalised, so as a rule of thumb, you should use indices on columns that will be frequently used in where or join statements.   

## Primary Key
The primary key is a field or combination of fields that uniquely identifies a row in a table. There can only be one primary key in a table. The primary key values must be unique and Not nullable.

## Syntax in create statement

You can use the create statement to define the Primary Key and the indices in your table. Let us see it with an example:

```sql
CREATE TABLE Person (
    PersonId INT NOT NULL AUTO_INCREMENT, -- Normal Integer. At most 4294967296 different PersonIds. 
    Name VARCHAR(30) DEFAULT '', -- Name with at most 30 characters. Default empty string.
    IsAwesome TINYINT(1) DEFAULT 0, -- Either 0 (False) or 1 (True). Default 0    
    AwesomenessLevel DECIMAL(13,3) DEFAULT 12.3, -- Decimal value with 13 digits of precision, up to 10^10 and with three digits after the decimal point. Defaults to 10.3
    Gender ENUM('Male', 'Female', 'Non-Binary') DEFAULT 'Male', -- Either male, female, or non binary. Default 'Male'
    Created TIMESTAMP DEFAULT CURRENT_TIMESTAMP, -- timestamp with the date of creation, defaults to current timestamp
    PRIMARY KEY (PersonId),
    INDEX current_timestamp_idx (Created)
    );
```

In the example, we have used two statements in the *PersonID* column to ensure that it is a valid identifier:  ```NOT NULL``` to make it non nullable (no row with null PersonId is ever allowed), and ```AUTOINCREMENT``` to auto increment the value of the identifier automatically. 
Below, we have used the statement ```PRIMARY KEY (PersonId)``` to make it the primary key. 

The statement ```INDEX current_timestamp_idx (Created)``` creates a new index in the column Created. This will speed up search statements that use this column in the WHERE statement. 

 
In database engines like MySQL ```KEY``` is a synonym for ```INDEX```. In others like Oracle, they are not equivalent.  

 
 **IMPORTANT**
 Since the PersonId is now auto incremented by the engine, you should not include it in the field list in INSERT statements, as the engine will assign a value automatically.
 
## Primary Key with Alter Table statement
 You can also add a primary key to a table using the ```ALTER``` statement:
 
 ```sql
ALTER TABLE Person
    ADD PRIMARY KEY (PersonId);
```

If you want to name the primary key or use several fields as primary key use:

```sql
ALTER TABLE Person
    ADD CONSTRAINT PersonId_PK PRIMARY KEY (PersonId);
```

**IMPORTANT**
If you use ALTER TABLE to add a primary key, the primary key column(s) must have been declared to not contain NULL values (when the table was first created). 
## Index Syntax with Create Index statement
 You can also create an index on an existing table using the ```CREATE INDEX``` statement
 
```sql
# Create an index named gender_isawesome_idx on Person
CREATE INDEX gender_isawesome_idx
ON Person (Gender, IsAwesome);
```

## Shorthand in MySQL CREATE TABLE statement
The following syntax is also valid in MySQL:
```SQL
CREATE TABLE Person (
    PersonId INT PRIMARY KEY, -- Normal Integer. At most 4294967296 different PersonIds. 
    Name VARCHAR(30) DEFAULT '', -- Name with at most 30 characters. Default empty string.
    IsAwesome TINYINT(1) DEFAULT 0, -- Either 0 (False) or 1 (True). Default 0    
    AwesomenessLevel DECIMAL(13,3) DEFAULT 12.3, -- Decimal value with 13 digits of precision, up to 10^10 and with three digits after the decimal point. Defaults to 10.3
    Gender ENUM('Male', 'Female', 'Non-Binary') DEFAULT 'Male', -- Either male, female, or non binary. Default 'Male'
    Created TIMESTAMP DEFAULT CURRENT_TIMESTAMP, -- timestamp with the date of creation, defaults to current timestamp
    INDEX current_timestamp_idx (Created)
    );
```

By adding the PRIMARY KEY keywords in the definition of the PersonId field, MySQL adds the PRIMARY KEY constraint, Not nullabe condition and uniqueness restriction and in this sense it is a shorthand to define the Primary key. 


## Drop Primary Key
To drop a primary key use:

```sql
ALTER TABLE Person
    DROP PRIMARY KEY;
```

If you used a named constraint to create the primary key use instead:

```sql
ALTER TABLE Person
    DROP CONSTRAINT PersonId_PK;
```

## Foreign Keys
Foreign keys are used to explicitly define in the model the relations in our data. Foreign keys will prevent users from creating inconsistencies when deleting data.
Foreign keys also make join operations more efficient since the engine has more context information to optimise internal data search operations.

Let us create the following table Order:

```sql
CREATE TABLE Order (
    OrderId INT NOT NULL AUTO_INCREMENT,
    OrderNumber int NOT NULL,
    Person_fk int,
    
    PRIMARY KEY (OrderID),
    CONSTRAINT Order_Person_fk FOREIGN KEY (Person_fk) REFERENCES Person(PersonId)
);
```
The column ```Person_fk``` stores the relation (1:N) between Person and Order, specifying which person placed each order. It references the primary key of the "Person" table, "PersonId". 
This relationship is specified using the ```FOREIGN KEY``` statement in the definition of a constraint.
In this data model, if users try to delete a product referenced by an order in the Order table, the database engine will raise an error to prevent inconsistent data in the  order table.

You can also use the ```ALTER TABLE``` statement to add foreign key constraints:

```sql
ALTER TABLE Order
    ADD CONSTRAINT Order_Person_fk FOREIGN KEY (Person_fk) REFERENCES Person(PersonId);
```

## Drop a Foreign key
You can use the ```DROP CONSTRAINT``` statement to drop a foreign key constraint:
```sql
ALTER TABLE Order
    DROP CONSTRAINT Order_Person_fk;
```
# Join clauses
Join clauses are used to combine data from different tables. There are four different types of join clauses, described in the sections below.
### Inner join
The ```INNER JOIN``` clause selects records in both tables that meet the ```ON``` condition and the conditions in the ```WHERE``` clause, if present.

The syntax of the Inner join is:

```sql
SELECT column_name(s)
FROM table1
INNER JOIN table2
ON table1.column_name = table2.column_name;
```

For example, the clause, 

```sql
SELECT p.Name, o.OrderNumber
FROM Person p
INNER JOIN Order o
ON p.PersonId = o.Person_fk
```
Will return the names of all persons that placed an order together with the corresponding order numbers. If any person in the table did not place any order, it will not be present in the result.

### Left join
The ```LEFT JOIN``` clause selects all records from the left table that meets the criteria in the ```WHERE``` clause, if present, together with the selected fields from the right table.

The syntax of the Left join is:

```sql
SELECT column_name(s)
FROM table1
LEFT JOIN table2
ON table1.column_name = table2.column_name;
```

For example, the clause, 

```sql
SELECT p.Name, o.OrderNumber
FROM Person p
LEFT JOIN Order o
ON p.PersonId = o.Person_fk
```
Will return the names of all persons, together with the corresponding order numbers. If any person in the table placed more than one order, is name will appear repeated the number of order he or she placed. 

### Right join
The ```right JOIN``` clause selects all records from the right table that meets the criteria in the ```WHERE``` clause, if present, together with the selected fields from the left table.

The syntax of the Right join is:

```sql
SELECT column_name(s)
FROM table1
RIGHT JOIN table2
ON table1.column_name = table2.column_name;
```

For example, the clause, 

```sql
SELECT p.Name, o.OrderNumber
FROM Order o
RIGHT JOIN Person p
ON p.PersonId = o.Person_fk
```

will have the same result as the previous example with ```LEFT JOIN```. 

### Outer join
The ```Outer JOIN``` clause selects all records from the left table and all records from the right table that meet the criteria in the ```WHERE``` clause, if present.

The syntax of the Outer join is:

```sql
SELECT column_name(s)
FROM table1
FULL OUTER JOIN table2
ON table1.column_name = table2.column_name;
```

For example, the clause, 

```sql
SELECT p.Name, o.OrderNumber
FROM Person p
FULL OUTER JOIN Order o
ON p.PersonId = o.Person_fk
```
Will return the names of all persons, together with the corresponding orders they placed, if by any chance there were orders without a foreign key to Person (for instance self placed orders), they would appear too. 



